# Intro to API Testing

Why test? It's a prereq on evey job application and required by every set of production code ever. It's used to make sure services and programs not only do what is expected of them, but also to make sure the code is performant under stress. If that's not enough reason to, chances are, you're already doing it - just manually.

## So let's build an automated testing suite for this Soil API.

It starts with defining the services. Our client is a GUI (React as of Oct 28, 2025.) When a useer clicks on a part of the map, they want to access the soil data at that location. We're going to use [OpenAPI Specification (OAS)](https://spec.openapis.org/oas/latest.html), as well as an example or 
two to define what the client

Using the [API Definitions](./defining_the_api.ipynb), it's time to come up with some testing.

## GET soil?lat=number&lon=number

This is what our backend does:
1. Call the NRCS database
  ```
  curl -X POST "https://sdmdataaccess.nrcs.usda.gov/Tabular/post.rest" \
    -H "Content-Type: application/x-www-form-urlencoded" \
    -d "query=SELECT mup.mupolygonkey, mup.mukey, mup.mupolygongeo
  FROM mupolygon AS mup
  WHERE mup.mukey IN (
    SELECT mukey
    FROM SDA_Get_Mukey_from_intersection_with_WktWgs84('POINT(-122.449871 37.492633)')
  )" \
    -d "format=json"
  ```
2. Filter out the soil data to get one polygon.

So what test cases do we need to watch out for? There's a few types, but the most important are: Unit Testing, End-to-End testing, Load and Security testing, as well as Contract and Functional Testing.

### Unit Testing
Handling optional parameters correctly, returns appropriate errors, etc. We need to make sure that we've tested:
  1. (Behavior) A single polygon is returned for the coordinate when successful.
  2. (Error Handling) A 404 is sent when we call the api and no soil data is found.
    - lat and lon is in range, but the NRCS returns {}
  3. (Error Handling) A 400 is sent on invalid lat and lon
    - lat [-90, 90] and lon [-180, 180] are not in that range (save the NRCS db call).
    - lat and lon specified are not floats (save the NRCS db call)

### Security Testing 
Some items we need to test for:
  1. SQL Injection
    - `lat` and `lon` must be sanitized before going into the SQL query. This can be caught be the unit tests, but make sure a cheeky `); DROP TABLE;` returns a 400 status code.

### End-to-End testing
E2E will be done on the frontend for v1, since we really only have one call currently.

### Load Testing
A benchmark for the web service on a **MOCK** NRCS Soil DB instance will be used to check how many concurrent users we can have for a single instance of our service.



# References

Postman, “What is API Testing? A Guide to Testing APIs,” Postman API Platform, 2025. [Online]. Available: https://www.postman.com/api-platform/api-testing/. [Accessed: Oct. 28, 2025].

